# Тема "Методы сбора и обработки данных из сети Интернет"

### Урок 4. Парсинг HTML. XPath

---
### Задание 1

Написать приложение, которое собирает основные новости с сайта на выбор: **news.mail.ru**, **lenta.ru**, **yandex-новости**.

Для парсинга использовать **XPath**.

Структура данных должна содержать:
- название источника;
- наименование новости;
- ссылку на новость;
- дата публикации.

Сложить собранные новости в БД

Минимум один сайт, максимум - все три

In [2]:
import requests
from lxml import html
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError as dke
from pprint import pprint

In [3]:
from datetime import timedelta, datetime

In [4]:
client = MongoClient('127.0.0.1', 27017)
db = client['ya_news']

In [5]:
url = 'https://yandex.ru/news/?utm_source=main_stripe_big'

In [6]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36 Edg/94.0.992.31'}

In [8]:
rubrics = ['personal_feed', 'computers', 'science']

In [11]:
response = requests.get(url, headers=headers)  # params=params, proxies=proxies, если необходимо

In [12]:
dom = html.fromstring(response.text)

all_news = []

for rubric in rubrics:
    
    if rubric in db.list_collection_names():
        news_collection = db.get_collection(rubric)
    else:
        news_collection = db.create_collection(rubric)

        items = dom.xpath(f"//a[contains(@href,'rubric={rubric}') and @class='mg-card__link']/ancestor::article")

    for item in items:
        news = {}
        title = item.xpath(".//h2[@class='mg-card__title']/text()")[0].replace('\xa0', ' ')
        annotation = item.xpath(".//div[@class='mg-card__annotation']/text()")[0].replace('\xa0', ' ')
        link = item.xpath(".//h2[@class='mg-card__title']/../@href")[0]
        posting_time = item.xpath(".//span[@class='mg-card-source__time']/text()")[0]
        source = item.xpath(".//a[@class='mg-card__source-link']/text()")[0]

        print(title)
                
        if len(posting_time) == 5:
            date_time = str(datetime.today().date()) + ' ' + posting_time
        elif posting_time.startswith('вчера'):
            date_time = str(datetime.today().date() - timedelta(1)) + ' ' + posting_time.split()[2]
        else: 
            date_time = posting_time
        
        print(date_time, source)
        print(annotation)
        print(link)
        print()

        news['title'] = title
        news['date_time'] = date_time
        news['source'] = source
        news['annotation'] = annotation
        news['link'] = link

        try:
            news_collection.insert_one(news)
            news_collection.create_index('link', unique=True)
        except dke:
            continue

ДНК фекалий показала, что древние люди Австрии 2700 лет назад пили пиво и ели голубой сыр
2021-10-14 16:38 FBM.ru
Полученные результаты являются первым доказательством того, что люди производили голубой сыр в Европе железного века почти 2700 лет назад.
https://yandex.ru/news/story/DNK_fekalij_pokazala_chto_drevnie_lyudi_Avstrii_2700_let_nazad_pili_pivo_i_eli_goluboj_syr--052ae9afc185e989cfc4a368ecf286af?lang=ru&rubric=science&fan=1&stid=apwi5AsTuDls17PA6R2T&t=1634219349&persistent_id=166620483

Исследователь Арно Висмар: запах путресцина вызвал тревогу у респондентов
2021-10-14 16:27 Телеканал 360°
Исследователь Арно Висмар из школы психологии Университета Кента в Великобритании смог предположить, какой запах ассоциируется со смертью.
https://yandex.ru/news/story/Issledovatel_Arno_Vismar_zapakh_putrescina_vyzval_trevogu_urespondentov--e3353553a0359f29d54f6678ca923243?lang=ru&rubric=science&fan=1&stid=NpKpYFCebPfLsdhTGcCU&t=1634219349&persistent_id=166640033

Студентка БФУ имени Канта н